In [6]:
import time
import numpy as np
import qiskit
from qiskit.opflow import X,Z,I
from qiskit.opflow.state_fns import StateFn, CircuitStateFn

from modules.utils import *

import matplotlib.pyplot as plt

from scipy import sparse
import scipy.sparse.linalg.eigen.arpack as arp
%matplotlib inline

In [7]:
import numpy as np
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki
from qiskit.circuit import Parameter
from qiskit import BasicAer

I am basically following this https://qiskit.org/documentation/tutorials/operators/01_operator_flow.html#Part-IV:-Converters

In [44]:
L=5
anti = -1
gx = 1e-2
gz = 1e-2

In [45]:
two_qubit_H2 = QHIsing(L,anti,np.float32(gx),np.float32(gz))

In [99]:
two_qubit_H2 = 1 * (X^X^X^Z^X + Z^Z^Z^Z^X)
two_qubit_H2

PauliSumOp(SparsePauliOp([[ True, False, False, False,  True, False,  True,  True,
                 True, False,  True,  True,  True, False,  True, False,
                False, False],
               [ True, False, False, False, False, False,  True,  True,
                 True, False,  True,  True,  True,  True,  True, False,
                False, False]],
              coeffs=[1.+0.j, 1.+0.j]), coeff=1.0)

In [100]:
evo_time = Parameter('θ')
evolution_op = (evo_time * two_qubit_H2).exp_i()

In [101]:
h2_measurement = StateFn(two_qubit_H2).adjoint()
#print(h2_measurement)

In [102]:
evo_and_meas = h2_measurement @ evolution_op @ Zero
#print(evo_and_meas)

In [103]:
trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evo_and_meas)

In [104]:
bound = trotterized_op.bind_parameters({evo_time: .5})

In [105]:
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)

In [106]:
evo_time_points = [1,2,3,4]
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: evo_time_points})

In [107]:
h2_trotter_expectations.eval()

array([-1.565e-15+0.j, -1.900e-16+0.j, -7.900e-17+0.j,  6.400e-17+0.j])

but if you rerun it with complex values it doesnt work because the trotterization takes use of the unitarity of the Ops.

In [41]:
# Ops
H = QHIsing(L,anti,np.float32(gx),np.float32(gz))
H_meas = StateFn(H).adjoint()
mag = QMag(L,anti)

In [66]:
qiskit.opflow.One^5

DictStateFn({'11111': 1}, coeff=1.0, is_measurement=False)

In [67]:
qiskit.opflow.VectorStateFn(qiskit.opflow.One^5)

VectorStateFn(DictStateFn({'11111': 1}, coeff=1.0, is_measurement=False), coeff=1.0, is_measurement=False)

In [69]:
asd = H_meas @ qiskit.opflow.One^5

In [70]:
asd.eval()

RecursionError: maximum recursion depth exceeded in comparison

In [22]:
evo_time = qiskit.circuit.Parameter('θ') # note that we will have to bind it laterevo_time = Parameter('θ')

In [23]:
evolution_op = (evo_time  * H).exp_i() #* 1j
# expH = suzi.convert(H)

In [24]:
evo_and_meas = H_meas @ evolution_op

In [25]:
trotterized_op = qiskit.opflow.evolutions.PauliTrotterEvolution(trotter_mode=qiskit.opflow.evolutions.Suzuki(order=2, reps=1)).convert(evo_and_meas)

In [26]:
bound = trotterized_op.bind_parameters({evo_time: .5})

In [32]:
bound.eval()

SummedOp([CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19b60fd0>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19a5c2e0>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19a65ac0>, coeff=-0.010099999776482581), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca199f62e0>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca199f8a90>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19a122b0>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19b63490>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19a2a2b0>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca199b2a90>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.q

In [33]:
diagonalized_meas_op = qiskit.opflow.PauliExpectation().convert(trotterized_op)

In [34]:
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: 1})

In [35]:
h2_trotter_expectations.eval()

SummedOp([SummedOp([CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19ccdb80>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca199a8a60>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19937220>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19941a30>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19952280>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca1995ba90>, coeff=-0.010099999776482581), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca1996d2b0>, coeff=1.0), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca198f7ac0>, coeff=-0.009999999776482582), CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fca19909190>, coeff=-0.009999999776482582)], coeff=1.0, abelian=False), SummedO

In [35]:
H.__class__, evolution_op.__class__

(qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp,
 qiskit.opflow.evolutions.evolved_op.EvolvedOp)

In [38]:
def imaginary_time_evolution(tau, L, gx, gz, anti=-1):
    H = QHIsing(L,anti,np.float32(gx),np.float32(gz))
    evo_time = qiskit.circuit.Parameter('tau') # note that we will have to bind it later
    evolution_op = (tau * 1j * H).exp_i()
    trotterized_op = qiskit.opflow.evolutions.PauliTrotterEvolution(trotter_mode=qiskit.opflow.evolutions.Suzuki(order=2, reps=1)).convert(H_meas @ evolution_op)
    return trotterized_op

In [39]:
trott_op = imaginary_time_evolution(tau=1, L=5, gx=1e-2, gz=1e-2, anti=-1)

In [40]:
trott_op.bind_param({evo_})

AttributeError: 'ComposedOp' object has no attribute 'bind_param'

In [27]:
bound = trott_op.bind_parameters({evo_time: 1})

In [28]:
bound.to_circuit().draw()

global phase: 0
     ┌────────────────────────────────┐┌────────────────────────────────┐┌───┐»
q_0: ┤ RZ(-0.00999999977648258*I*tau) ├┤ RX(-0.00999999977648258*I*tau) ├┤ X ├»
     ├────────────────────────────────┤├────────────────────────────────┤└─┬─┘»
q_1: ┤ RZ(-0.00999999977648258*I*tau) ├┤ RX(-0.00999999977648258*I*tau) ├──■──»
     ├────────────────────────────────┤├────────────────────────────────┤     »
q_2: ┤ RZ(-0.00999999977648258*I*tau) ├┤ RX(-0.00999999977648258*I*tau) ├─────»
     ├────────────────────────────────┤├────────────────────────────────┤     »
q_3: ┤ RZ(-0.00999999977648258*I*tau) ├┤ RX(-0.00999999977648258*I*tau) ├─────»
     ├───────────────────────────────┬┘├────────────────────────────────┤     »
q_4: ┤ RZ(-0.0100999997764826*I*tau) ├─┤ RX(-0.00999999977648258*I*tau) ├─────»
     └───────────────────────────────┘ └────────────────────────────────┘     »
«     ┌───────────────┐┌───┐                                                 »
«q_0: ┤ RZ(1.0*I*tau) ├┤ X ├─────────────────────────────────────────────────»
«     └───────────────┘└─┬─┘┌───┐┌───────────────┐┌───┐                      »
«q_1: ───────────────────■──┤ X ├┤ RZ(1.0*I*tau) ├┤ X ├──────────────────────»
«                           └─┬─┘└───────────────┘└─┬─┘┌───┐┌───────────────┐»
«q_2: ────────────────────────■─────────────────────■──┤ X ├┤ RZ(1.0*I*tau) ├»
«                                                      └─┬─┘└───────────────┘»
«q_3: ───────────────────────────────────────────────────■───────────────────»
«                                                                            »
«q_4: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«                                                                »
«q_0: ───────────────────────────────────────────────────────────»
«                                                                »
«q_1: ───────────────────────────────────────────────────────────»
«     ┌───┐                                                      »
«q_2: ┤ X ├──────────────────────────────────────────────────────»
«     └─┬─┘┌───┐┌───────────────┐┌───┐┌───┐┌───────────────┐┌───┐»
«q_3: ──■──┤ X ├┤ RZ(1.0*I*tau) ├┤ X ├┤ X ├┤ RZ(1.0*I*tau) ├┤ X ├»
«          └─┬─┘└───────────────┘└─┬─┘└─┬─┘└───────────────┘└─┬─┘»
«q_4: ───────■─────────────────────■────■─────────────────────■──»
«                                                                »
«                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_1: ────────────────────────────────────────────────────────────────────────»
«                   ┌───┐                       ┌───────────────┐        ┌───┐»
«q_2: ──────────────┤ X ├───────────────────────┤ RZ(1.0*I*tau) ├────────┤ X ├»
«                   └─┬─┘                       └───────────────┘        └─┬─┘»
«q_3: ────────────────■────────────────────────────────────────────────────■──»
«     ┌────────────────────────────────┐┌───────────────────────────────┐     »
«q_4: ┤ RX(-0.00999999977648258*I*tau) ├┤ RZ(-0.0100999997764826*I*tau) ├─────»
«     └────────────────────────────────┘└───────────────────────────────┘     »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«                   ┌───┐                       ┌───────────────┐         ┌───┐»
«q_1: ──────────────┤ X ├───────────────────────┤ RZ(1.0*I*tau) ├─────────┤ X ├»
«                   └─┬─┘                       └───────────────┘         └─┬─┘»
«q_2: ────────────────■─────────────────────────────────────────────────────■──»
«     ┌────────────────────────────────┐┌────────────────────────────────┐     »
«q_3: ┤ RX(-0.00999999977648258*I*tau) ├┤ RZ(-0.009

In [29]:
mag_res = ~StateFn(mag) @ StateFn(trott_op)

In [30]:
mag_res.eval()

TypeError: ParameterExpression with unbound parameters ({Parameter(tau)}) cannot be cast to a float.

In [31]:
bound.eval()

TypeError: ParameterExpression with unbound parameters ({Parameter(tau)}) cannot be cast to a float.

In [ ]:
bound = trotterized_op.bind_parameters({evo_time: tau})

As far as I can tell, we need to work with these StateFns instead of circuits

In [11]:
# this is a dummy circuit for the sake of checking how I can combine the evolved state with a circuit later
circ = qiskit.circuit.library.TwoLocal(5,"rx","cz", reps=1,entanglement="linear")

In [12]:
circ.draw()

┌──────────┐   ┌──────────┐                                    
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├────────────────────────────────────
     ├──────────┤ │ └──────────┘┌──────────┐                        
q_1: ┤ RX(θ[1]) ├─■──────■──────┤ RX(θ[6]) ├────────────────────────
     ├──────────┤        │      └──────────┘┌──────────┐            
q_2: ┤ RX(θ[2]) ├────────■───────────■──────┤ RX(θ[7]) ├────────────
     ├──────────┤                    │      └──────────┘┌──────────┐
q_3: ┤ RX(θ[3]) ├────────────────────■───────────■──────┤ RX(θ[8]) ├
     ├──────────┤                                │      ├──────────┤
q_4: ┤ RX(θ[4]) ├────────────────────────────────■──────┤ RX(θ[9]) ├
     └──────────┘                                       └──────────┘

In [13]:
circ_fn = CircuitStateFn(circ)

In [14]:
print(circ_fn)

CircuitStateFn(
     ┌──────────┐   ┌──────────┐                                    
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├────────────────────────────────────
     ├──────────┤ │ └──────────┘┌──────────┐                        
q_1: ┤ RX(θ[1]) ├─■──────■──────┤ RX(θ[6]) ├────────────────────────
     ├──────────┤        │      └──────────┘┌──────────┐            
q_2: ┤ RX(θ[2]) ├────────■───────────■──────┤ RX(θ[7]) ├────────────
     ├──────────┤                    │      └──────────┘┌──────────┐
q_3: ┤ RX(θ[3]) ├────────────────────■───────────■──────┤ RX(θ[8]) ├
     ├──────────┤                                │      ├──────────┤
q_4: ┤ RX(θ[4]) ├────────────────────────────────■──────┤ RX(θ[9]) ├
     └──────────┘                                       └──────────┘
)


In [15]:
evolved_processed = evolution_op @ circ_fn # first the imaginary time evolution, then the dummy processing
print(evolved_processed )

ComposedOp([
  e^(-i*1.0*I*t * (
    1.0 * ZZIII
    - 0.009999999776482582 * XIIII
    - 0.010099999776482581 * ZIIII
    + 1.0 * IZZII
    - 0.009999999776482582 * IXIII
    - 0.009999999776482582 * IZIII
    + 1.0 * IIZZI
    - 0.009999999776482582 * IIXII
    - 0.009999999776482582 * IIZII
    + 1.0 * IIIZZ
    - 0.009999999776482582 * IIIXI
    - 0.009999999776482582 * IIIZI
    - 0.009999999776482582 * IIIIX
    - 0.009999999776482582 * IIIIZ
  )),
  CircuitStateFn(
       ┌──────────┐   ┌──────────┐                                    
  q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├────────────────────────────────────
       ├──────────┤ │ └──────────┘┌──────────┐                        
  q_1: ┤ RX(θ[1]) ├─■──────■──────┤ RX(θ[6]) ├────────────────────────
       ├──────────┤        │      └──────────┘┌──────────┐            
  q_2: ┤ RX(θ[2]) ├────────■───────────■──────┤ RX(θ[7]) ├────────────
       ├──────────┤                    │      └──────────┘┌──────────┐
  q_3: ┤ RX(θ[3]) ├───────

In [16]:
trotterized_op = qiskit.opflow.evolutions.PauliTrotterEvolution(trotter_mode=qiskit.opflow.evolutions.Suzuki(order=2, reps=1)).convert(evolved_processed)
print(trotterized_op)

CircuitStateFn(
global phase: 0
     ┌──────────┐   ┌──────────┐┌──────────────────────────────┐»
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├┤ RZ(-0.00999999977648258*I*t) ├»
     ├──────────┤ │ └──────────┘└─────────┬──────────┬─────────┘»
q_1: ┤ RX(θ[1]) ├─■──────■────────────────┤ RX(θ[6]) ├──────────»
     ├──────────┤        │                └──────────┘          »
q_2: ┤ RX(θ[2]) ├────────■─────────────────────■────────────────»
     ├──────────┤                              │                »
q_3: ┤ RX(θ[3]) ├──────────────────────────────■────────────────»
     ├──────────┤                                               »
q_4: ┤ RX(θ[4]) ├───────────────────────────────────────────────»
     └──────────┘                                               »
«     ┌──────────────────────────────┐                                »
«q_0: ┤ RX(-0.00999999977648258*I*t) ├────────────────────────────────»
«     ├──────────────────────────────┤┌──────────────────────────────┐»
«q_1: ┤ RZ(-0.009999999776

In [17]:
bound = trotterized_op.bind_parameters({evo_time: .5j})

In [18]:
bound.to_circuit().draw()

┌──────────┐   ┌──────────┐┌─────────────────────────┐»
q_0: ┤ RX(θ[0]) ├─■─┤ RX(θ[5]) ├┤ RZ(0.00499999988824129) ├»
     ├──────────┤ │ └──────────┘└───────┬──────────┬──────┘»
q_1: ┤ RX(θ[1]) ├─■──────■──────────────┤ RX(θ[6]) ├───────»
     ├──────────┤        │              └──────────┘       »
q_2: ┤ RX(θ[2]) ├────────■───────────────────■─────────────»
     ├──────────┤                            │             »
q_3: ┤ RX(θ[3]) ├────────────────────────────■─────────────»
     ├──────────┤                                          »
q_4: ┤ RX(θ[4]) ├──────────────────────────────────────────»
     └──────────┘                                          »
«     ┌─────────────────────────┐                           »
«q_0: ┤ RX(0.00499999988824129) ├───────────────────────────»
«     ├─────────────────────────┤┌─────────────────────────┐»
«q_1: ┤ RZ(0.00499999988824129) ├┤ RX(0.00499999988824129) ├»
«     └───────┬──────────┬──────┘├─────────────────────────┤»
«q_2: ────────┤ RX(θ[7]) ├───────┤ RZ(0.00499999988824129) ├»
«             └──────────┘       └───────┬──────────┬──────┘»
«q_3: ─────────────■─────────────────────┤ RX(θ[8]) ├───────»
«                  │                     ├──────────┤       »
«q_4: ─────────────■─────────────────────┤ RX(θ[9]) ├───────»
«                                        └──────────┘       »
«                ┌───┐            ┌────────────────────────┐┌───┐     »
«q_0: ───────────┤ X ├────────────┤ RZ(-0.500000000000000) ├┤ X ├─────»
«                └─┬─┘            └────────────────────────┘└─┬─┘┌───┐»
«q_1: ─────────────■──────────────────────────────────────────■──┤ X ├»
«     ┌─────────────────────────┐                                └─┬─┘»
«q_2: ┤ RX(0.00499999988824129) ├──────────────────────────────────■──»
«     ├─────────────────────────┤┌─────────────────────────┐          »
«q_3: ┤ RZ(0.00499999988824129) ├┤ RX(0.00499999988824129) ├──────────»
«     ├─────────────────────────┤├─────────────────────────┤          »
«q_4: ┤ RZ(0.00504999988824129) ├┤ RX(0.00499999988824129) ├──────────»
«     └─────────────────────────┘└─────────────────────────┘          »
«                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────»
«     ┌────────────────────────┐┌───┐                                         »
«q_1: ┤ RZ(-0.500000000000000) ├┤ X ├─────────────────────────────────────────»
«     └────────────────────────┘└─┬─┘┌───┐┌────────────────────────┐┌───┐     »
«q_2: ────────────────────────────■──┤ X ├┤ RZ(-0.500000000000000) ├┤ X ├─────»
«                                    └─┬─┘└────────────────────────┘└─┬─┘┌───┐»
«q_3: ─────────────────────────────────■──────────────────────────────■──┤ X ├»
«                                                                        └─┬─┘»
«q_4: ─────────────────────────────────────────────────────────────────────■──»
«                                                                             »
«                                                                        »
«q_0: ───────────────────────────────────────────────────────────────────»
«                                                                        »
«q_1: ───────────────────────────────────────────────────────────────────»
«                                                                        »
«q_2: ───────────────────────────────────────────────────────────────────»
«     ┌────────────────────────┐┌───┐┌───┐┌────────────────────────┐┌───┐»
«q_3: ┤ RZ(-0.500000000000000) ├┤ X ├┤ X ├┤ RZ(-0.500000000000000) ├┤ X ├»
«     └────────────────────────┘└─┬─┘└─┬─┘└────────────────────────┘└─┬─┘»
«q_4: ────────────────────────────■────■──────────────────────────────■──»
«                                                                        »
«                                                                »
«q_0: ───────────────────────────────────────────────────────────»
«                    

In [20]:
# ED result
Smag = Mag(L,anti)
init_state, E, ham = ising_groundstate(L, anti, np.float(gx), np.float(gz))
Sen = E
Smags = init_state.T.conj()@Smag@init_state #Magnetization with Numpy results

<ipython-input-20-100ad986e4f8>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  init_state, E, ham = ising_groundstate(L, anti, np.float(gx), np.float(gz))
